<a href="https://colab.research.google.com/github/pushpalatha2297/hds5210-2023/blob/main/midterm/midterm-2023-extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [1]:
import requests

def find_hospital(age, sex, risk):
    '''
    This function takes the risk score, age, and sex of a patient and figures out which is the nearest facility where the person can be taken
    if they are in some emergency using the REST web service.

    Parameters:
    age (int): Age of the patient.
    sex (str): Gender of the patient ('male' or 'female').
    risk (float): Risk percentage as a numeric value between 0 and 1.

    Returns:
    str or None: The name of the hospital where the patient should be sent, or None if there's an error.
    '''

    try:
        url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        hospital_name = data.get('hospital')
        return hospital_name
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def summarize_results(patients):
    """
    Summarize patient allocation based on PRIEST probability with an additional business rule.

    This function takes a dictionary of patient data generated by the `process_people` function
    and applies a new business rule: If the PRIEST probability is less than 10%, change the
    hospital and address to both have a value of 'Home' indicating patients with a lower
    probability of an adverse outcome can just go home rather than be admitted to a hospital.
    It then summarizes the results and counts how many patients are being sent to each location.

    Args:
    patients (dict): A dictionary where each key is a patient's unique identifier (patient_number)
                     and the value is a list containing patient information.

    Returns:
    dict: A summary dictionary where the keys represent the location (hospital or home)
          and the values represent the count of patients allocated to each location.

    Example:
    {
        "Home": 123,
        "Hospital A": 10,
        "Hospital B": 20,
        "Hospital C": 30
    }
    """
    summary = {'Home': 0}
    for patient_data in patients.values():
        # Extract patient information including PRIEST probability
        sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status, risk_score = patient_data

        # Check if PRIEST probability is less than 10%
        if risk_score < 0.1:
            # Change hospital and address to 'Home'
            patient_data[-2] = 'Home'
            patient_data[-3] = 'Home'
            summary['Home'] += 1
        else:
            # Send patient to the hospital
            hospital_name = find_hospital(age, sex, risk_score)  # Use the find_hospital function to get the hospital name
            if hospital_name in summary:
                summary[hospital_name] += 1
            else:
                summary[hospital_name] = 1

    return summary

# Test case 1: Create a sample patients dictionary (replace with actual data)
patients_1 = {
    'patient1': ['male', 35, 18, 0.92, 70, 120, 37, 'alert', 'air', 'unrestricted normal activity', 0.05],
    'patient2': ['female', 45, 22, 0.94, 75, 125, 36.5, 'alert', 'air', 'unrestricted normal activity', 0.08],
    'patient3': ['male', 60, 19, 0.91, 80, 130, 37.2, 'alert', 'air', 'unrestricted normal activity', 0.09],
}

# Sample patient data with a mix of PRIEST probabilities
patients_2 = {
    'patient4': ['female', 65, 28, 0.90, 95, 140, 38.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care', 0.15],
    'patient5': ['male', 75, 24, 0.93, 85, 135, 36.8, 'alert', 'air', 'unrestricted normal activity', 0.06],
    'patient6': ['female', 55, 20, 0.92, 75, 130, 37.1, 'alert', 'air', 'unrestricted normal activity', 0.12],
}

# Sample patient data with high PRIEST probabilities
patients_3 = {
    'patient7': ['male', 45, 19, 0.91, 70, 115, 36.5, 'alert', 'air', 'unrestricted normal activity', 0.18],
    'patient8': ['female', 50, 30, 0.89, 100, 140, 38.2, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care', 0.25],
    'patient9': ['male', 70, 22, 0.93, 80, 130, 37.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care', 0.22],
}

# Call the summarize_results function with test data
result_1 = summarize_results(patients_1)
result_2 = summarize_results(patients_2)
result_3 = summarize_results(patients_3)

# Print the results
print(result_1)
print(result_2)
print(result_3)

{'Home': 3}
{'Home': 1, 'Wesley Woods Geriatric Hospital': 1, 'Select Specialty Hospital - Northeast Atlanta': 1}
{'Home': 0, 'Southwest Hospital and Medical Center': 1, 'Select Specialty Hospital - Northeast Atlanta': 1, 'Wesley Woods Geriatric Hospital': 1}
